In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import inspect
import sys
from pandas import read_csv

In [3]:
sys.path.insert(0, 'code')

In [4]:
from validation_utils import compare, get_accuracy, get_test_data
from preprocessing import get_words_from_text, get_words_with_freq_from_text, file_filter

### Load real data

In [5]:
PATH_TO_VOCABULARY = "data/text.txt"
TEST_DATA_PATH = "data/aspell-60_6-normal.txt"

In [6]:
test_data = get_test_data(TEST_DATA_PATH, "\t")
freq_vocab = get_words_with_freq_from_text(PATH_TO_VOCABULARY)

# Słowo bez kontekstu

`Oto sformalizowany zapis problemu w tym przypadku:`
\begin{equation*}
\underset{c\in C}{\operatorname{argmax}} P(c \mid w)   
\end{equation*}

`gdzie` $c$ `jest kandydatem na poprawną wersję napotkanego słowa` $w$ , `a` $C$ `jest zbiorem kandydatów który dalej nazywany bedzie słownikiem. `

`Widząc sam wyraz` $w$ `nie jesteśmy w stanie nic powiedzieć o tym warunkowym prawdopodobieństwie.
Na szczęście z pomocą może przyjść nam Bayes:`

\begin{equation*}
P(c\mid w) = \frac{P(w \mid c) * P(c)}{P(w)}\\
\end{equation*}

`Dlatego ze` $ P(w) $ `jest wspólne dla każdego` ${c\in C}$ `to wyżej wspomniany sformalizowany problem możemy zapisać jako:`

\begin{equation*}
\underset{c\in C}{\operatorname{argmax}} P(c \mid w) = \underset{c\in C}{\operatorname{argmax}} P(w \mid c) * P(c)
\end{equation*}

## Naive Approches

`Naiwność podejść bierze się stąd że zakładamy że każdy ze słów w słowniku ma takie samo prawdopodobieństwo wystąpienia` $P(c) = \frac{1}{|C|}$ `oraz tego że literówka transformuje nam słowo w wyraz z poza słownika (jako że nie rozważamy kontekstu jest to całkiem odpowiednie założenie).
Tak więc wzór redukuje nam się do:`

\begin{equation*}
\underset{c\in C}{\operatorname{argmax}} P(c \mid w) = \underset{c\in C}{\operatorname{argmax}} P(w \mid c)
\end{equation*}

In [7]:
from naive_approach import NaiveApproach

In [8]:
naive = NaiveApproach(path_to_vocab=PATH_TO_VOCABULARY)

In [9]:
print(inspect.getsource(naive.scan_and_compare))

    def scan_and_compare(self, word):
        if word in self.vocab:
            return [word]
        candidates = ["xxxx"]
        d_min = edit_distance(candidates[0], word, transpositions=True)
        for c in self.vocab:  # c stands candidate
            d = edit_distance(c, word, transpositions=True)
            if d < d_min:
                candidates = [c]
                d_min = d
            elif d == d_min:
                candidates.append(c)
        return candidates



`Sensownym wydaje się też założyć że jeżeli popełniono literówkę to nie jest ona zaburzeniem jednocześnie pierwszej i ostatniej pozycji.
Tak więc:`

\begin{equation*}
\underset{c\in C}{\operatorname{argmax}} P(c \mid w) = \underset{c\in C}{\operatorname{argmax}} P(w, (F(c,w)=0 \cup  L(c,w)=0) \mid c)
\end{equation*}

`gdzie` $F$ `i` $L$ `to odległość między odpowiednio pierwszymi i ostatnimi znakami podanych słów`

In [10]:
print(inspect.getsource(naive.scan_and_compare_check_first_and_last))

    def scan_and_compare_check_first_and_last(self, word):
        if word in self.vocab:
            return [word]
        candidates = ["xxxx"]
        d_min = edit_distance(candidates[0], word, transpositions=True)
        for c in self.vocab:
            if word[0] == c[0] or word[-1] == c[-1]:
                d = edit_distance(c, word, transpositions=True)
                if d < d_min:
                    candidates = [c]
                    d_min = d
                elif d == d_min:
                    candidates.append(c)
        return candidates



`Następne podejście umożliwia nie przeglądanie całego słownika, dając szybciej wyniki, ale jedynie takie które są odległe o co najwyżej 2 od` $w$, `w metryce L-D.`<br>
`Kandydaci są generowani w obrębie` $w$, `a następnie wyniki są filtrowane przez słownik. Zbyt mały słownik nie uchwyci pożądanego słowa, a zbyt duży zwróci wielu kandydatów.`

In [11]:
print(inspect.getsource(naive.generate_and_scan))

    def generate_and_scan(self, word):
        candidates = list(known_generated(word, self.vocab))
        if candidates:
            return candidates
        return ["xxxx"]



### Benchmark

In [ ]:
%%time
get_accuracy(naive.scan_and_compare, test_data, freq_vocab)

In [14]:
%%time
get_accuracy(naive.scan_and_compare_check_first_and_last, test_data, freq_vocab)

Ratio of OOV:  0.1276747503566334
CPU times: user 2min 9s, sys: 781 ms, total: 2min 9s
Wall time: 8min 48s


0.6657156443176415

In [12]:
%%time
get_accuracy(naive.generate_and_scan, test_data, freq_vocab)

Ratio of OOV:  0.1276747503566334
CPU times: user 2min 30s, sys: 1.27 s, total: 2min 31s
Wall time: 2min 53s


0.6678554446029482

`Założenie zaimplementowane w drugim modelu nie wydaje się ograniczające, wydaje się wręcz być żadnym ograniczeniem, gdyż wynik jest taki sam, a czas 7.5 razy szybszy.`<br>
`Ostatni model redukujący przestrzeń kandydatów jest jednak najszybszy (~23 razy szybszy od pierwszego modelu), i ma nieco lepszy wynik, ale jest to kwestia kolejkowania się kandydatów w liście najbardziej prawdopodobnych słów.`

## Vocab with Frequencies

`We wcześniejszym podpunkcie zakładaliśmy takie samo prawdopodobieństwo dla każdego kandydata.`<br>
`W tym nareszcie podejdziemy do sprawy empirycznie. Bedziemy na podstawie odpowiednio dużego tekstu zliczać wystąpienia danego kandydata i uważać stosunek liczby jego wystąpień do liczby wszystkich wyrazów jako estymacje jego prawdopodobieństwa.`

\begin{equation*}
\underset{c\in C}{\operatorname{argmax}} P(c \mid w) = \underset{c\in C}{\operatorname{argmax}} P(w \mid c) * P(c)
\end{equation*}

In [13]:
from words_with_freq_approach import FrequencyApproach

In [14]:
frequnecy = FrequencyApproach(path_to_vocab=PATH_TO_VOCABULARY)

In [15]:
print(inspect.getsource(frequnecy.scan_and_compare))  

    def scan_and_compare(self, word):
        if word in self.vocab:
            return [word]
        candidates = []
        p_max = 0
        w_len = len(word)

        for c in self.vocab:  # c stands candidate
            d = edit_distance(c, word, transpositions=True)
            if d < w_len:
                p = self.vocab[c]  * (1 - (math.log(d) / math.log(w_len))) # przemyśl to
                if p_max < p:
                    p_max = p
                    candidates = [c]
                elif p_max == p:
                    candidates.append(c)
        if candidates:
            return candidates
        else:
            return [word]



In [16]:
print(inspect.getsource(frequnecy.scan_and_compare_check_first_and_last))

    def scan_and_compare_check_first_and_last(self, word):
        if word in self.vocab:
            return word, 0
        candidates = []
        p_max = 0
        w_len = len(word)
        for c in self.vocab:
            if word[0] == c[0] or word[-1] == c[-1]:
                d = edit_distance(c, word, transpositions=True)
                if d < w_len:
                    p = self.vocab[c]  * (1 - (math.log(d) / math.log(w_len))) # przemyśl to
                    if p_max < p:
                        p_max = p
                        candidates = [c]
                    elif p_max == p:
                        candidates.append(c)
        if candidates:
            return candidates
        else:
            return [word]



In [17]:
print(inspect.getsource(frequnecy.peter_norvig_approach))

    def peter_norvig_approach(self, word):
        if word in self.vocab:
            return [word]
        p_max = 0
        approx = []
        candidates = known_generated(word, self.vocab)
        for c in candidates:
            p = self.vocab[c]
            if p > p_max:
                p_max = p
                approx = [c]
            elif p == p_max:
                approx.append(c)
        if approx:
            return approx
        else:
            return [word]



### Benchmark

In [ ]:
%%time
get_accuracy(frequnecy.scan_and_compare, test_data, freq_vocab)

In [21]:
%%time
get_accuracy(frequnecy.scan_and_compare_check_first_and_last, test_data, freq_vocab)

Ratio of OOV:  0.1276747503566334
CPU times: user 2min 9s, sys: 825 ms, total: 2min 10s
Wall time: 8min 51s


0.009747979077508321

In [20]:
%%time
get_accuracy(frequnecy.peter_norvig_approach, test_data, freq_vocab)

Ratio of OOV:  0.1276747503566334
CPU times: user 2min 29s, sys: 1.13 s, total: 2min 30s
Wall time: 2min 49s


0.7063718497384689

`Pierwsze dwa modele mają jednak dość poważny problem, który nie został dotychczas jawnie wspomniany, ale jest dość nasuwający się.`

`Nie znamy rozkładu generatora, przez co nie znamy` $P(w \mid c)$,  `w powyższych dwóch modelach wzór na jego rozkład wziął się przecież znikąd.`<br>
`Wcześniej nie musielismy się tym przejmować dlatego że wyrazy najbliższe zostały uważane za te mające największe prawdopodobieństo, nie musieliśmy znać ilościowego charakteru tego rozkładu. W tym przypadku całkowity rozkład nie zależy już tylko od rozkładu generatora, ale też od częstości występowania kandydata w języku, zależy więć od dwóch estymatorów` $P(w \mid c), P(c)$.

`Założenie o bliskości kandydata względem obserwowanego wyrazu wydaje się na tyle rozsądne by uzyskać zadowalający wynik, dlatego podejście Petera Norviga działa. Słowa w tym podejściu klastetyzują się względem metryki D-L i wybierana jest grupa o najmniejszej odległości, następnie spośród tej grupy wybierane jest najczęsciej występujące słowo.`<br>
`Podejście użyte w kodzie numer 3 można zapisać jako:`

\begin{equation*}
\underset{c\in C}{\operatorname{argmax}} P(c \mid w) = \underset{c\in C}{\operatorname{argmax}}(\underset{c\in C}{\operatorname{argmax}} P(w \mid c)) * P(c)
\end{equation*}




# Load syntetic data

`Przypuśćmy jednak że chcieliśmy napisać "thin", ale napisaliśmy "thinn", w tym przypadku lepszym kandydatem dla ostatniego modelu jest "think", jako wyraz częściej występujący w słowniku.`

In [36]:
freq_vocab["thin"] < freq_vocab["think"]

True

`By temu zapobiec możemy skorzystać ze sposobu posługiwania się danym wyrazem w języku. Możemy skorzystać z informacji zawartej w kontekście danego słowa`

## Context Approches

`W tym podejściu wybieramy wyraz, na podstawie obserwacji wyrazu i jego kontekstu.`

\begin{equation*}
\underset{c\in C}{\operatorname{argmax}} P(c \mid w_{1},w_{2})
\end{equation*}

`gdzie` $w_{2}$ `jest wyrazem który zamierzamy sprawdzić, a` $w_{1}$ `wyrazem tworzącym kontekst`

\begin{equation*}
\\\underset{c\in C}{\operatorname{argmax}} P(c \mid w_{1},w_{2}) = \underset{c\in C}{\operatorname{argmax}} \frac{P(c, w_{1},w_{2})}{P(w_{1},w_{2})} = \underset{c\in C}{\operatorname{argmax}} \frac{P(w_{2} \mid w_{1}, c) * P(c \mid w_{1})}{P(w_{2} \mid w_{1})} = \\\underset{c\in C}{\operatorname{argmax}} P(w_{2} \mid w_{1}, c) * P(c \mid w_{1})
\end{equation*}

In [18]:
from context_approach_C import ContextApproach

In [19]:
PATH_TO_BI_GRAMS = "data/bigrams_final_encoded.pkl"
PATH_TO_TRI_GRAMS = "data/trigrams_final_encoded.pkl"
PATH_TO_BI_GRAMS_TEST_DATA = "data/bi_grams_test_data.csv"
PATH_TO_TRI_GRAMS_TEST_DATA = "data/tri_grams_test_data.csv"

In [7]:
context_bi = ContextApproach(load_bi_grams_path=PATH_TO_BI_GRAMS)

In [20]:
context_tri = ContextApproach(load_tri_grams_path=PATH_TO_TRI_GRAMS)

In [8]:
#load tri grams test data
bi_grams_test_data = read_csv(PATH_TO_BI_GRAMS_TEST_DATA).values.tolist()[:10000]
reduced_uni_grams_test_data_bi = context_bi.reduce_gram_test_data(bi_grams_test_data)

In [21]:
#load tri grams test data
tri_grams_test_data = read_csv(PATH_TO_TRI_GRAMS_TEST_DATA).values.tolist()[:10000]
reduced_bi_grams_test_data_tri = context_tri.reduce_gram_test_data(tri_grams_test_data)
reduced_uni_grams_test_data_tri = context_tri.reduce_gram_test_data(reduced_bi_grams_test_data_tri)

In [23]:
print(inspect.getsource(context_tri.context_approach_bigrams))

    def context_approach_bigrams(self, utterance):
        w1, w2 = word_tokenize(utterance)
        c_based_on_context = self.bi_grams[w1]
        if c_based_on_context:
            vocab = {c for c in c_based_on_context}
            candidates = known_generated(w2, vocab)
            if candidates:
                p_max = 0
                approx = []
                for c in candidates:
                    p = c_based_on_context[c]
                    if p_max < p:
                        p_max = p
                        approx = [c]
                    elif p_max == p:
                        approx.append(c)
                return approx

        return ["xxxxx"]



In [24]:
print(inspect.getsource(context_tri.context_approach_trigrams))

    def context_approach_trigrams(self, utterance):
        w1, w2, w3 = word_tokenize(utterance)
        c_based_on_context = self.tri_grams[w1][w2]
        if c_based_on_context:
            vocab = {c for c in c_based_on_context}
            candidates = known_generated(w3, vocab)
            if candidates:
                p_max = 0
                approx = []
                for c in candidates:
                    p = c_based_on_context[c]
                    if p_max < p:
                        p_max = p
                        approx = [c]
                    elif p_max == p:
                        approx.append(c)
                return approx
            
        return ["xxxxx"]



## Benchmark - TODO (lambda's pickle problem)

In [ ]:
%%time
vocab = context_bi.get_vocab_for_bigram()
get_accuracy(context_bi.context_approach_bigrams, bi_grams_test_data, vocab, threads=4)

In [ ]:
%%time
vocab = context_tri.get_vocab_for_trigram(tri_grams)
get_accuracy(context_tri.context_approach_trigrams, tri_grams_test_data, c, vocab)

## Syntetic Benchmark

`Porównywana będzie niestety tylko skuteczność z powodu problemów ze zrównoleglaniem działań przy użyciu defaultdicta w którym zgromadziliśmy ngramy.`<br>
`Modele jednak róznią się tylko dostępem do większego słownika o drzewiastej strukturze.`

In [30]:
from tqdm import tqdm

### Context

#### Trigram

In [31]:
%%time
counter = 0
for pair in tqdm(tri_grams_test_data):
    if pair[1] in context_tri.context_approach_trigrams(pair[0])[: 2]:
        counter += 1
print(counter/10000)

100%|██████████| 10000/10000 [08:38<00:00, 19.30it/s]

0.2808
CPU times: user 8min 33s, sys: 6.03 s, total: 8min 39s
Wall time: 8min 38s


##### YAYKS !!!
`Zbyt mały zbiór tri-gramów, niestety większy nie mieści nam się w pamięci`

#### Bigram

In [32]:
%%time
counter = 0
for pair in tqdm(bi_grams_test_data):
    if pair[1] in context_bi.context_approach_bigrams(pair[0])[: 2]:
        counter += 1
print(counter/10000)

100%|██████████| 10000/10000 [10:01<00:00, 16.62it/s]

0.7858
CPU times: user 9min 55s, sys: 7.69 s, total: 10min 2s
Wall time: 10min 1s


In [39]:
%%time
counter = 0
for pair in tqdm(reduced_bi_grams_test_data_tri):
    if pair[1] in context_bi.context_approach_bigrams(pair[0])[: 2]:
        counter += 1
print(counter/10000)

100%|██████████| 10000/10000 [10:45<00:00, 15.49it/s]

0.8582
CPU times: user 10min 38s, sys: 8.26 s, total: 10min 46s
Wall time: 10min 45s


In [9]:
vocab_bi = context_bi.get_vocab_for_bigram()

In [25]:
vocab_tri = context_tri.get_vocab_for_trigram()

### Frequency

In [21]:
%%time
get_accuracy(frequnecy.peter_norvig_approach, reduced_uni_grams_test_data_bi, vocab_bi)

Ratio of OOV:  0.0004
CPU times: user 6min 14s, sys: 2.98 s, total: 6min 17s
Wall time: 6min 39s


0.6069

In [26]:
%%time
get_accuracy(frequnecy.peter_norvig_approach, reduced_uni_grams_test_data_tri, vocab_tri)

Ratio of OOV:  0.0014
CPU times: user 3min 52s, sys: 1.87 s, total: 3min 53s
Wall time: 5min 4s


0.6662

### Naive

In [22]:
%%time
get_accuracy(naive.generate_and_scan, reduced_uni_grams_test_data_bi, vocab_bi)

Ratio of OOV:  0.0004
CPU times: user 6min 18s, sys: 3.24 s, total: 6min 22s
Wall time: 6min 44s


0.4048

In [27]:
%%time
get_accuracy(naive.generate_and_scan, reduced_uni_grams_test_data_tri, vocab_tri)

Ratio of OOV:  0.0014
CPU times: user 3min 53s, sys: 1.97 s, total: 3min 55s
Wall time: 5min 10s


0.4633

# Wnioski

`Przy prawdopodobieństwie warunkowym` $ P(w \mid c) $ `uważamy wszystkie słowa` $"c"$ `o tej samej odległosci od` $"w"$ `za tak samo prawdopodobne. Ale nie wydaje się to być sprawiedliwe.`

#### Oto przykład:
`Niech` $w = abot$, `kandydatami równie prawdopodobnymi są słowa` $about$, $abbot$ `jak i` $bot$. `Opuszczenie` $"u"$ `albo drugiego` $"b"$ `wydaje się jednak bardziej prawdopodobne niż dodanie` $"a"$ `odległej na klawiaturze od` $"b"$ `o doraźną liczbę klawiszy.`
`Z drugiej strony` $"a"$ `jest jednym z przedimków. Ktoś mógł chcieć napisać` $a$ $bot$, `ale zapomniał spacji, albo wstawił spację przed` $"a"$.

`Rozkład rozmieszczenia literówek w napisie nie wydaje się również jednostajny, z tego co zaobserwowaliśmy na rzeczywistych danych, wśród słów nie zauważono literówek różniących się jednocześnie na pierwszym i ostatnim miejscu w słowie.
Pokazuje to złożoność rozkładu generatora literówek. Systemy regułowe wydają się być najeżone wyjątkami w swoim podejściu do wyłapywania błędów.`<br>
`Spróbujmy więc stworzyć narzędzie które będzie uczyło się tego rozkładu na podstawie danych - zbudujmy i wytrenujmy do tego celu sieć neuronową.`

#### Tak jak wspomnieliśmy na wstępie, sieć będzie uczyła się jedynie na danych syntetycznych z powodu braku odpowiednio dużej liczby otagowanych danych rzeczywistych.
